In [1]:
import pandas as pd, numpy as np

In [3]:
df = pd.read_excel("Adhoc Task/Sampling Amit.xlsx")
df

,Lead Owner,State,ORG,Lead Source,Staff,Created Date,HD Name,HD Mobile,OPSCO of HD,District,Citizen Name,Citizen Mobile,Scheme Name,Helper
0,PID/DBSBA/2023/DI/0266,ANDHRA PRADESH,AP - DBS Bank Foundation,27 Aug to 02 Sep,NaN,2023-08-04,Nirukattu Sunila,9392125744,Vuppala Raju,Nandyal,Goothi Resma bi,9998846404,Jagananna Amma Vodi _AP,NaN
1,PID/DBSBA/2023/DI/0266,ANDHRA PRADESH,AP - DBS Bank Foundation,27 Aug to 02 Sep,NaN,2023-08-25,Neerukattu Vanitha,9676273930,Vuppala Raju,Nandyal,Jangati Sandhyarani,9989981544,Jagananna Amma Vodi _AP,NaN
2,PID/DBSBA/2023/DI/0266,ANDHRA PRADESH,AP - DBS Bank Foundation,27 Aug to 02 Sep,NaN,2023-08-24,Sugali Varalakshmi Bai,9966867225,Vuppala Raju,Nandyal,Usharanibai Sm,9989882394,Jagananna Amma Vodi _AP,NaN
3,PID/DBSBA/2023/DI/0266,ANDHRA PRADESH,AP - DBS Bank Foundation,27 Aug to 02 Sep,NaN,2023-08-25,Kukutla Lakshmi Devi,6281795775,Vuppala Raju,Nandyal,Talari Lakshmi Prasanna,9989882231,Loan for SHG Women at 4 % interest_AP,NaN
4,PID/DBSBA/2023/DI/0266,ANDHRA PRADESH,AP - DBS Bank Foundation,27 Aug to 02 Sep,NaN,2023-08-31,Appikonda Sanyasamma,8008234830,Vuppala Raju,Anakapalli,Karri Laxmamma,9989875618,Pradhan Mantri Suraksha Bima Yojna (PMSBY)_AP,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,PID/NASSC/2022/TL/0225,UTTARAKHAND,UT - Nasscom Foundation,27 Aug to 02 Sep,NaN,2023-08-26,Suhel Ahamd,7668187071,Himanshu Saini,HARIDWAR,Jaheera Jaheera,6395913739,Aadhaar linking to Voter ID (Central),NaN
25403,PID/NASSC/2022/TL/0225,UTTARAKHAND,UT - Nasscom Foundation,27 Aug to 02 Sep,NaN,2023-08-26,Mamta Verma,9759338591,Renu Sharma,HARIDWAR,Krishnaa Krishnaa,6395762326,Aadhaar linking to Voter ID (Central),NaN
25404,PID/NASSC/2022/TL/0225,UTTARAKHAND,UT - Nasscom Foundation,27 Aug to 02 Sep,NaN,2023-08-27,Poonam Saini,8218651075,Himanshu Saini,HARIDWAR,Rohit Kumar,6395542122,Digital Health ID card (Central),NaN
25405,PID/NASSC/2022/TL/0225,Uttarakhand,UT - Nasscom Foundation,27 Aug to 02 Sep,NaN,2023-06-13,Rakhi,6395330350,Himanshu Saini,Haridwar,Reeta Reeta,6395174972,E-Shram Card (Central),NaN


In [6]:
samp = df.sample(frac=0.2)

In [7]:
samp.to_csv("C:\\Python\\export\\sample.csv", index = False)